In [ ]:
from keras.models import load_model
!pip install mtcnn
# confirm mtcnn was installed correctly
import mtcnn
# print version
print(mtcnn.__version__)

import numpy as np
from mtcnn.mtcnn import MTCNN
from numpy import asarray
#!pip install pycocotools
import io
import os
from pycocotools.coco import COCO
from PIL import Image
import matplotlib.pyplot as plt


# Simple python package to shut up Tensorflow warnings and logs.
!pip install silence_tensorflow
import silence_tensorflow.auto

In [ ]:
#set up folders for extracted faces and the corresponding captions 
# initialize COCO API for instance annotations
import os
dataDir = './'

image_folder = "extracted_eyes"

savedirimage = dataDir +  image_folder + "/"


if not os.path.exists(savedirimage):
	os.makedirs(savedirimage)

In [ ]:
# extract eyes from extracted_face_image
def extract_eyes_from_extratced_face(filename, index, required_size = (300,100),left_eye_corner = 20, right_eye_corner = 230):
    # load image from file
    image = Image.open(filename)
    # convert to RGB, if needed
    image = image.convert('RGB')
    # convert to array
    pixels = np.asarray(image)
    # create the detector, using default weights
    detector = MTCNN()
    # detect faces in the image
    results = detector.detect_faces(pixels)
    if len(results)>=1:
        left_right_eye_x_diff = results[0]['keypoints']['right_eye'][0] - results[0]['keypoints']['left_eye'][0] 
        left_right_eye_y_diff = abs(results[0]['keypoints']['right_eye'][1] - results[0]['keypoints']['left_eye'][1]) 
    else:
        return False
    # extract the bounding box from the first face
    if left_right_eye_x_diff>=100 and left_right_eye_y_diff < 8:
        #print(results[0])
        xeye = left_eye_corner
        yeye = int((results[0]['keypoints']['left_eye'][1]+results[0]['keypoints']['right_eye'][1])/2 -10)
        width = right_eye_corner
        height = int((yeye+results[0]['keypoints']['nose'][1])/2)
        eyes = pixels[yeye:height, xeye:width]
        # resize pixels to the model size
        image = Image.fromarray(eyes)
        image = image.resize(required_size)
        eyes_array = np.asarray(image)
        #plt.imshow(eyes_array)
        plt.imsave(os.path.join(savedirimage, 'eyes_img_{}.jpg'.format(index)),eyes_array)
        return True
    return False

In [ ]:
files = os.listdir('../extracted_faces_from_COCO/')

In [ ]:
len(files)

In [ ]:
index =0
for file in files:
    if extract_eyes_from_extratced_face(os.path.join('../extracted_faces_from_COCO/',file), index = index):
        print(index)
        index+=1